In [1]:
import message_db
import sqlite3
import re

### Connect Config

In [2]:
con, cur = message_db.connect()

In [ ]:
#con.close()

### Query Table Counts

In [ ]:
tables = ['chat', 'message', 'handle']

count_rows = lambda table_name: cur.execute('SELECT COUNT(*) AS count FROM {}'.format(table_name)).fetchone()['count']
table_counts = {table: count_rows(table) for table in tables}

In [ ]:
table_counts

### Reading SQL query file

In [ ]:
fd = open('messages.sql', 'r')
sql_file = fd.read()
fd.close()

In [ ]:
sql_commands = [re.sub('\n\s{0}', ' ', cmd) for cmd in sql_file.split(';') if cmd is not '']

### Chat Table Queries

SELECTs all chats and builds a dictionary of chat_id --> chat_info

In [ ]:
chat_query = 'SELECT ROWID AS id, account_id, service_name AS service FROM chat'

In [ ]:
cur.execute(chat_query)
chat_result = cur.fetchall()

In [ ]:
chats = {c['id']: c for c in chat_result}

### Handle Table

Selects all handles builds a dictionary mapping handle_ids to handle info (phone number)

In [ ]:
handle_query = 'SELECT ROWID AS id, id AS full_number, country, service, uncanonicalized_id AS local_number, person_centric_id AS person_id FROM handle'

In [ ]:
cur.execute(handle_query)
handle_result = cur.fetchall()

In [ ]:
handles = {h['id']: h for h in handle_result}

### Messages Table

Selects all messages and builds a list to be used for bucketing of messages to their corresponding chats / handles

In [ ]:
msg_query = "SELECT ROWID AS id, text, is_from_me, datetime((date/1000000000) + strftime('%s', '2001-01-01 00:00:00'), 'unixepoch', 'localtime') AS date FROM message ORDER BY date DESC"

In [ ]:
cur.execute(msg_query)
message_result = cur.fetchall()

In [ ]:
messages = {m['id']: m for m in message_result}

In [ ]:
messages['113849']

### Chat - Message Join Table

In [ ]:
cmj_query = "SELECT cmj.message_id AS message_id, cmj.chat_id AS chat_id, text, is_from_me, handle_id, datetime((date/1000000000) + strftime('%s', '2001-01-01 00:00:00'), 'unixepoch', 'localtime') AS date FROM chat_message_join AS cmj INNER JOIN message AS m ON cmj.message_id = m.ROWID ORDER BY date"

In [ ]:
cur.execute(cmj_query)
cmj_result = cur.fetchall()

### Chat - Handle Join Table

In [ ]:
chj_query = "SELECT chj.handle_id AS handle_id, chj.chat_id AS chat_id, id AS full_number, country, service, uncanonicalized_id AS local_number, person_centric_id AS person_id FROM chat_handle_join AS chj INNER JOIN handle AS h ON chj.handle_id = h.ROWID ORDER BY handle_id"

In [ ]:
cur.execute(chj_query)
chj_result = cur.fetchall()

### Chat - Handle - Message Query

#### NOTE
1. some messages have duplicates as iMessage/SMS in chat_message_join
2. some messages (presumably attachments) have NULL text values

In [8]:
combined_query = "SELECT m.ROWID AS message_id, cmj.chat_id AS chat_id, m.handle_id AS handle_id, h.id AS phone_number, text, is_from_me, style AS chat_style, m.service AS service, group_id, person_centric_id AS persion_id, datetime((date/1000000000) + strftime('%s', '2001-01-01 00:00:00'), 'unixepoch', 'localtime') AS date FROM message AS m INNER JOIN chat_message_join AS cmj ON cmj.message_id = m.ROWID INNER JOIN chat AS c ON cmj.chat_id = c.ROWID LEFT JOIN handle AS h ON m.handle_id = h.ROWID WHERE text IS NOT NULL GROUP BY message_id"
cur.execute(combined_query)
full_messages = cur.fetchall()

In [11]:
len(full_messages)

247770